# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5]

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking", axis=1)

# Review the features DataFrame
X.head(5)

### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.columns)

# Review the number of features
number_input_features

In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
number_output_neurons = 1


In [ ]:
# Create the Sequential model instance
nn = tf.keras.models.Sequential()

# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [ ]:
# Display the Sequential model summary
nn.summary()

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model using 50 epochs and the training data
nn.fit(X_train_scaled, y_train, epochs=50)

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Export your model to a keras file
nn.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")


# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
nn_model_predictions = nn_imported.predict(X_test_scaled)

# Display a sample of the predictions
nn_model_predictions[:5]

In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
df = pd.DataFrame(nn_model_predictions, columns=["predictions"])
df["predictions"] = df["predictions"].apply(lambda x: round(x, 0))
df

### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, df["predictions"]))

---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1.**
To build a recommendation system for student loan options, it is essential to collect data that directly impacts loan suitability and affordability. This includes demographic information like age, location, and educational level, which helps identify eligibility for specific loans. Loan preferences, such as interest rate type, repayment terms, and loan type, provide insights into the student's priorities. Academic details, including the institution, field of study, and expected graduation date, are crucial for matching loans aligned with their educational goals. Financial history, such as credit score, income level, existing debts, and payment history, is vital to determine eligibility and repayment capacity. Additionally, behavioral data, like past interactions with loan options or applications, allows the system to refine recommendations based on user patterns. Collecting this data ensures the system can make personalized and relevant recommendations tailored to each student's unique financial and academic circumstances.

**2.**
The model would primarily use content-based filtering because the data collected is specific to individual students, including their preferences, financial history, and demographics. These features allow the system to recommend loans that closely match the characteristics and needs of the user. Collaborative filtering may not be ideal in this scenario because student loan needs can be highly individualized, and there may not be sufficient shared preferences among users to build recommendations effectively.

**3.**
Data Privacy and Security: Handling sensitive financial and personal information requires robust data protection measures. Breaches or misuse of this data could lead to severe legal and ethical consequences, eroding user trust.

Bias in Recommendations: If the training data contains biases (e.g., historical trends favoring certain demographics), the system may unintentionally propagate these biases, leading to unfair or discriminatory recommendations. Addressing fairness and ensuring equitable access is critical for building trust and compliance with regulations.